Se añade la carpeta correctamente source.


In [1]:
# Se importan las librerías que se van a usar
import requests
import time
import re
import pandas as pd
import os
import urllib.parse
#!pip install webdriver-manager
import chromedriver_binary 
from bs4 import BeautifulSoup
from selenium.webdriver import Firefox
from webdriver_manager.firefox import GeckoDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from tqdm import tqdm
from urllib.parse import urlparse

In [22]:
# url de partida para obtener la información de las películas
#url = "https://www.imdb.com/chart/top"
url = "https://www.imdb.com/feature/genre"

# Se modifica el user-agent para evitar no ser bloqueados
headers = {
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,\
*/*;q=0.8",
"Accept-Encoding": "gzip, deflate, sdch, br",
"Accept-Language": "en-US,en;q=0.8",
"Cache-Control": "no-cache",
"dnt": "1",
"Pragma": "no-cache",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36\
    (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"
}

# url login
#url_login = "https://www.imdb.com/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.imdb.com%2Fregistration%2Fap-signin-handler%2Fimdb_us&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=imdb_us&openid.mode=checkid_setup&siteState=eyJvcGVuaWQuYXNzb2NfaGFuZGxlIjoiaW1kYl91cyIsInJlZGlyZWN0VG8iOiJodHRwczovL3d3dy5pbWRiLmNvbS9jaGFydC90b3A_cmVmXz1sb2dpbiJ9&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&tag=imdbtag_reg-20"
url_login = "https://www.imdb.com/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.imdb.com%2Fregistration%2Fap-signin-handler%2Fimdb_us&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=imdb_us&openid.mode=checkid_setup&siteState=eyJvcGVuaWQuYXNzb2NfaGFuZGxlIjoiaW1kYl91cyIsInJlZGlyZWN0VG8iOiJodHRwczovL3d3dy5pbWRiLmNvbS9mZWF0dXJlL2dlbnJlLz9yZWZfPWxvZ2luIn0&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&tag=imdbtag_reg-20"
# Parámetros que son necesarios para hacer el login
data_login =  {
    'email':'luckigt@icloud.com',
    'password':'Password'
}
# Requests y el parser del html
response = requests.get(url,headers=headers)
soup = BeautifulSoup(response.text,"html.parser")


In [102]:

# Películas
soup_names = soup.select('td.titleColumn a')

# Sacar nombre de las películas
movies_names = [names.get_text() for names in soup_names]


In [65]:
# Año de las peliculas
#soup_movies = soup.select('td.titleColumn span[class=secondaryInfo]')
soup_years = soup.find_all('span', {'class' : 'secondaryInfo'})

years_of_film = [years.get_text() for years in soup_years]

# Eliminamos los parentesis del año del estreno de las películas
import re
years_of_film_clean = [re.sub("[()]","",years_cleaned) for years_cleaned in years_of_film]

In [68]:
# Ratings de las peliculas
soup_ratings = soup.find_all("td",class_='imdbRating')

ratings_of_film = [ratings.get_text() for ratings in soup_ratings]

#Eliminamos espacios a los lados
ratings_of_film_clean = [ratings_cleaned.strip() for ratings_cleaned in ratings_of_film]


In [5]:
# DRIVER CON FIREFOX

# Instalas este modulo con pip install webdriver-manager

# Abres el navegador Firefox
driver = Firefox(executable_path=GeckoDriverManager().install())
driver.get(url_login)

# Rellena email, password y hace click, y nos logamos
driver.find_element(By.ID,"ap_email").send_keys(data_login['email'])
driver.find_element(By.ID,"ap_password").send_keys(data_login['password'])
driver.find_element(By.ID,"signInSubmit").click()





[WDM] - Downloading: 19.0kB [00:00, 4.96MB/s]                   
/var/folders/_h/lw6vm6js40v053j5mjqh7fcw0000gn/T/ipykernel_5706/2778879890.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Firefox(executable_path=GeckoDriverManager().install())


In [ ]:
# Creas la session con request
#session = requests.Session()


#login = session.post(url_login,data = data_login,headers=headers)

#for i in login.history:
#    if i.status_code == 402:
#        print("Error!")

In [21]:
# Get the urls of the genres of the films
genres = soup.find_all(attrs={'class': 'image'}) 
urls_genres = [x.get('href') for genre in genres  for x in genre.findChildren('a')]

# Get the names of the genres of the films
names_genres = [x.get('alt') for genre in genres  for x in genre.findChildren('img')]

names_genres

['Comedy',
 'Sci-Fi',
 'Horror',
 'Romance',
 'Action',
 'Thriller',
 'Drama',
 'Mystery',
 'Crime',
 'Animation',
 'Adventure',
 'Fantasy',
 'Comedy-Romance',
 'Action-Comedy',
 'Superhero']

In [26]:
# Get all the text of the page identified by the input URL
response = requests.get(url,headers=headers)
soup = BeautifulSoup(response.text,"html.parser")

# Get all the text of the page identified by the input URL
genres = soup.find_all(attrs={'class': 'image'}) 

# Get the urls of the genres of the films
urls_genres = [x.get('href') for genre in genres  for x in genre.findChildren('a')]

# Get the names of the genres of the films
names_genres = [x.get('alt') for genre in genres  for x in genre.findChildren('img')]

# Create the dict_genres
dict_genres = {names_genres[i]: urls_genres[i] for i in range(len(names_genres))}
dict_genres

{'Comedy': 'https://www.imdb.com/search/title?genres=comedy&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=3396781f-d87f-4fac-8694-c56ce6f490fe&pf_rd_r=NK3CZ9827P3B9WEDPFZN&pf_rd_s=center-1&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr1_i_1',
 'Sci-Fi': 'https://www.imdb.com/search/title?genres=sci-fi&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=3396781f-d87f-4fac-8694-c56ce6f490fe&pf_rd_r=NK3CZ9827P3B9WEDPFZN&pf_rd_s=center-1&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr1_i_2',
 'Horror': 'https://www.imdb.com/search/title?genres=horror&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=3396781f-d87f-4fac-8694-c56ce6f490fe&pf_rd_r=NK3CZ9827P3B9WEDPFZN&pf_rd_s=center-1&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr1_i_3',
 'Romance': 'https://www.imdb.com/search/title?genres=romance&explore=title_type,genres&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e0da8c98-35e8-4ebd-8e86-e7d39c92730c&pf_rd_r=NK3CZ9827P3B9WEDPFZN&pf_rd_s=center-2&pf_rd_t=15051&pf_rd_i=genre&ref_=ft_gnr_pr2_i_1'